In [1]:
import numpy as np
import pathlib
from matplotlib import pyplot as plt
import pydpf
import torch
from data_prep import prepare_data
import model
import neural_networks
import training

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda_gen = torch.Generator(device=device).manual_seed(2)

## Prepare data

In [2]:
#prepare_data('cuda:0')

In [3]:
observation_encoding_size = 128
scaling = 1000.

In [4]:
def flatten_gens(list_of_gens):
    return [item for gen in list_of_gens for item in gen]
    
def is_in_it(item, it):
    return any(id(item) == id(item_) for item_ in it)
    

def get_SSM():
    encoder = neural_networks.ObservationEncoder(observation_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.3)
    decoder = neural_networks.ObservationDecoder(observation_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.3)
    state_encoder = neural_networks.StateEncoder(observation_encoding_size, generator=cuda_gen, dropout_keep_ratio=0.3)
    observation_partial_flows = [neural_networks.RealNVP_cond(dim = observation_encoding_size, hidden_dim=observation_encoding_size, condition_on_dim=observation_encoding_size, generator = cuda_gen), neural_networks.RealNVP_cond(dim = observation_encoding_size, hidden_dim=observation_encoding_size, condition_on_dim=observation_encoding_size, generator = cuda_gen)]
    flow_cov = torch.nn.Parameter(torch.eye(observation_encoding_size, device=device)*1000)
    observation_flow = neural_networks.NormalizingFlowModel_cond(pydpf.MultivariateGaussian(torch.zeros(observation_encoding_size, device=device), cholesky_covariance= flow_cov, diagonal_cov=True, generator=cuda_gen), observation_partial_flows, device)
    observation_model = model.MazeObservation(observation_flow, encoder, decoder, state_encoder)
    #observation_model = model.SimpleMazeObservation(encoder, decoder, state_encoder)
    dynamic_cov = torch.diag(torch.tensor([10/scaling, 10/scaling, 0.1], device=device))
    dynamic_model = model.MazeDynamic(cuda_gen, dynamic_cov)
    proposal_partial_flows = [neural_networks.RealNVP_cond(dim = 3, hidden_dim=32, condition_on_dim=observation_encoding_size, generator=cuda_gen, zero_i=True), neural_networks.RealNVP_cond(dim = 3, hidden_dim=32, condition_on_dim=observation_encoding_size, generator=cuda_gen, zero_i=True)]
    proposal_flow = neural_networks.NormalizingFlowModel_cond(None, proposal_partial_flows, device)
    proposal_model = model.MazeProposal(proposal_flow, dynamic_model)
    prior_model = model.MazePrior(2*1000/scaling, 1.3*1000/scaling, cuda_gen)
    #prior_model = model.MazePriorCheat()
    encoder_parameters = flatten_gens([encoder.parameters(), state_encoder.parameters(), decoder.parameters()])
    flow_parameters = flatten_gens([observation_flow.parameters(), proposal_flow.parameters(), prior_model.parameters()])
    #SSM = pydpf.FilteringModel(dynamic_model=dynamic_model, proposal_model=proposal_model, prior_model=prior_model, observation_model=observation_model)
    SSM = pydpf.FilteringModel(dynamic_model=dynamic_model, prior_model=prior_model, observation_model=observation_model)
    return SSM, encoder_parameters, flow_parameters, [flow_cov]
            

In [5]:
def transform_control(control, **data):
    output = control/torch.tensor([[[scaling, scaling, 1.]]], device=device)
    return output
    

In [6]:
def normalise_obs(observation, **data):
    return (observation - torch.mean(observation))/torch.std(observation)
    

In [ ]:
SSM, encoder_params, flow_params, flow_cov = get_SSM()
dpf = pydpf.DPF(SSM=SSM, resampling_generator=cuda_gen)
#dpf = pydpf.OptimalTransportDPF(SSM=SSM, regularisation=0.1, step_size=0.75)
dpf.to(device)
opt = torch.optim.AdamW([{'params': encoder_params, 'lr': 0.01}, {'params': flow_params, 'lr': 0.01}], weight_decay=1e-4)
opt_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.95)
data = pydpf.StateSpaceDataset(data_path= pathlib.Path('.').parent.absolute().joinpath('data/maze_data.csv'), state_prefix='state', control_prefix='control', device = device)
data.apply(normalise_obs,  'observation')
scaling_tesnor = torch.tensor([[[scaling, scaling, 1.]]], device=device)
data.apply(lambda state, **data: (state - torch.tensor([[[1000., 650., 0.]]], device=device))/scaling_tesnor, 'state')
data.apply(transform_control, 'control')
print('Data Loaded')
training.train(dpf, opt, data, 100, (100, 1000, 1000), (64, 64, 32), (0.45, 0.2, 0.35), (10., 1., 2.), torch.Generator().manual_seed(0), None, 'MSE', 99, pre_train_epochs=0, device=device, lr_scheduler=opt_scheduler, state_scaling = scaling)


Data Loaded
epoch 1/100, train loss: 14.752500415378147, validation position RMSE: 1519.5460769585106, validation angle RMSE: 1.823504470079161


In [1]:
obs = data.observation[:, 1000:1001]
state = data.state[:, 1000:1001]
print(state[0])
control = data.control[:, 1000:1001]
time = 99
batch = 0
encoded_obs = SSM.observation_model.encoder(obs.reshape(obs.size(0)*obs.size(1), 3, 24, 24))
dpf.update()
with torch.inference_mode():
    ps = dpf(1000, 99, {'state': pydpf.State(), 'weight': pydpf.Weight()}, observation=encoded_obs.reshape(100, 1, encoded_obs.size(1)).contiguous(), ground_truth=state, control=control)
numpy_ps = ps['state'].detach().cpu().numpy()

numpy_weight = ps['weight'].detach().cpu().numpy()
print(numpy_weight[0])
numpy_weight_norm = (np.exp(numpy_weight) / np.sum(np.exp(numpy_weight), axis=-1, keepdims=True))[:,0,:]
#print(numpy_weight[0, 0, :])
plt.scatter(numpy_ps[time,batch,:,0]*scaling, numpy_ps[time,batch,:,1]*scaling, alpha=np.exp(numpy_weight[time, batch, :])/np.max(np.exp(numpy_weight[time, batch, :])))
plt.scatter(state.cpu().numpy()[time,batch,0]*scaling, state.cpu().numpy()[time,batch,1]*scaling)
plt.scatter(np.sum(numpy_ps[time,batch,:,0]*numpy_weight_norm[time], keepdims=True)*scaling, [np.sum(numpy_ps[time,batch,:,1]*numpy_weight_norm[time], keepdims=True)*scaling], color='red')
plt.xlim(-1000, 1000)
plt.ylim(-650, 650)
plt.show()

NameError: name 'data' is not defined